In [5]:
import geopandas as gpd
import pandas as pd
import fiona

In [20]:
file = r'C:\Users\Ari\Desktop\Fall2020\5092\week1\lab1.gpkg'
# List all the available layers, prepare lists of polygons and tables
layers = fiona.listlayers(r'C:\Users\Ari\Desktop\Fall2020\5092\week1\lab1.gpkg')
soils = [f for f in layers if f.startswith('soil')]
tables = [f for f in layers if f.startswith('muag')]

print(layers)

['soilmu_a_co001', 'soilmu_a_co618', 'soilmu_a_co641', 'soilmu_a_co642', 'soilmu_a_co643', 'soilmu_a_co644', 'soilmu_a_co645', 'soilmu_a_co651', 'soilmu_a_co653', 'wbdhu8_lab1', 'muaggatt_co001', 'muaggatt_co618', 'muaggatt_co641', 'muaggatt_co642', 'muaggatt_co643', 'muaggatt_co644', 'muaggatt_co645', 'muaggatt_co651', 'muaggatt_co653']


In [21]:
joined_features = []
for poly in soils:
    for table in tables:
        if poly[-5:] == table[-5:]: # Grab the layers that match
            # Read each table and polygon
            poly_fc = gpd.read_file(file, layer=poly)
            table_fc = gpd.read_file(file, layer=table)
            table_fc = table_fc.drop(columns='geometry')
            # Join
            joined = pd.merge(poly_fc, table_fc, left_on='MUSYM', right_on='musym')
            
            # Add new field
            joined['mapunit'] = poly[-5:]
            
            # Append to the empyt list
            joined_features.append(joined)
            
            print(f'Done with {poly}')

Done with soilmu_a_co001
Done with soilmu_a_co618
Done with soilmu_a_co641
Done with soilmu_a_co642
Done with soilmu_a_co643
Done with soilmu_a_co644
Done with soilmu_a_co645
Done with soilmu_a_co651
Done with soilmu_a_co653


In [22]:
merged = pd.concat(joined_features)

In [23]:
# Print header of new geodataframe
merged.head()

,SPATIALVER,MUSYM,MUKEY,Shape_Length,Shape_Area,geometry,musym,aws025wta,aws0150wta,drclassdcd,mapunit
0,4.0,AaB,94035,0.010144,0.000004,"MULTIPOLYGON (((-104.42220 39.93400, -104.4248...",AaB,4.15,25.4,Well drained,co001
1,4.0,AaB,94035,0.011057,0.000004,"MULTIPOLYGON (((-104.27200 39.78590, -104.2749...",AaB,4.15,25.4,Well drained,co001
2,4.0,AaB,94035,0.010620,0.000005,"MULTIPOLYGON (((-104.34210 39.89100, -104.3428...",AaB,4.15,25.4,Well drained,co001
3,4.0,AaB,94035,0.015856,0.000013,"MULTIPOLYGON (((-104.28820 39.84590, -104.2850...",AaB,4.15,25.4,Well drained,co001
4,4.0,AaB,94035,0.025435,0.000025,"MULTIPOLYGON (((-104.28650 39.83190, -104.2835...",AaB,4.15,25.4,Well drained,co001


In [24]:
watershed = gpd.read_file(file, layer='wbdhu8_lab1')

In [25]:
watershed

,STATES,NAME,Shape_Length,Shape_Area,HUC8,geometry
0,CO,St. Vrain,2.675418,0.268070,10190005,"MULTIPOLYGON (((-105.52543 40.29793, -105.5234..."
1,CO,Big Thompson,2.734573,0.228711,10190006,"MULTIPOLYGON (((-105.31793 40.62803, -105.3147..."


In [26]:
# Intesect
intersected = gpd.overlay(merged, watershed, how='intersection')

In [27]:
# Grouped by HUC8
summarized = intersected.groupby(by='NAME').count()

In [28]:
for idx, row in summarized.iterrows():
    fc_count = row['SPATIALVER']
    print(f'{fc_count} features classes in watershed {idx}')

1979 features classes in watershed Big Thompson
2073 features classes in watershed St. Vrain
